<a href="https://colab.research.google.com/github/VielF/ColabProjects/blob/main/Pr%C3%A1tica_Filtragem_Espacial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade do Vale do Itajaí<br>
Escola Politécnica<br>
Processamento Digital de Sinais: Imagens

# Exercício Avalitivo de Filtragem Espacial

### Tutoriais da OpenCV

- https://docs.opencv.org/master/d9/df8/tutorial_root.html
- https://www.geeksforgeeks.org/opencv-python-tutorial/

## Conversão de imagem RGB em imagem Grayscale

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow


In [3]:
#abria a imagem
img = cv2.imread('../dataset/OperacaoPontuais_FiltroEspacial/data/t1.jpg',1)
#cv2_imshow(img)

In [4]:
#mostrando a imagem colorida
cv2.imshow('in', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#aplicando conversão básica
#numpy
#img_grayscale_basic = (img[ : , : ,0]+img[ : , : ,1]+img[ : , : ,2])/3

#cv2
B, G, R = cv2.split(img)
img_grayscale_basic = (B+G+R)/3
#converter a imagem em tons de cinza para o tipo uint8 garante que os valores dos pixels estejam dentro do intervalo 0 255
img_grayscale_basic = np.array(img_grayscale_basic, dtype=np.uint8)

cv2.imshow('img_grayscale_basic', img_grayscale_basic)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#aplicando conversão luminancia ponderada
#img_grayscale_basic = 0.299*img[ : , : ,0] + 0.587*img[ : , : ,1] + 0.114*img[ : , : ,2]

#cv2
B, G, R = cv2.split(img)
img_grayscale_pondered = 0.299*B+0.587*G+0.114*R

img_grayscale_pondered = np.array(img_grayscale_pondered, dtype=np.uint8)

cv2.imshow('img_grayscale_basic', img_grayscale_pondered)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Filtro Espacial


In [7]:
def gauss_create(sigma=1, size_x=3, size_y=3):
    '''
    Create normal (gaussian) distribuiton
    '''
    x, y = np.meshgrid(np.linspace(-1,1,size_x), np.linspace(-1,1,size_y))
    calc = 1/((2*np.pi*(sigma**2)))
    exp = np.exp(-(((x**2) + (y**2))/(2*(sigma**2))))
    
    return exp*calc
   

### Suavização

1. Implemente a sua própria convolução manualmente, em formato de função, utilizando os conceitos vistos em aula
2. Aplique a filtragem de suavização, com as máscaras 3x3, 5x5 e 7x7, com os filtros:
 - média
 - gaussino (escolha uma distribuição adequada, podendo utilizar a função acima ou recuperando a distribuição usada na OpenCV)
 - mediana 
3. Relate as observações visíveis nas imagens de saída com os filtros usados, descrevendo o comportamento visual em relação as bordas, ruídos e esmaecimento da imagem com os diferentes kernels

#### Dicas
 - Você pode adicionar ruído nas imagens usando a função random_noise: `from skimage.util import random_noise`. Cuidado com o retorno da função, talvez seja necessário regularizar a mesma novamente para o intervalo de 0 à 255.

In [82]:

def convolucao_manual_cinza(imagem, kernel):
    altura_imagem = imagem.shape[0]
    largura_imagem = imagem.shape[1]
    altura_kernel, largura_kernel = kernel.shape

    # Inicialize uma matriz para a imagem resultante
    imagem_resultante = np.zeros_like(imagem)

    # Calcule as margens para garantir que o kernel seja aplicado corretamente
    margem_vertical = altura_kernel // 2
    margem_horizontal = largura_kernel // 2

    # Realize a convolução
    for i in range(margem_vertical, altura_imagem - margem_vertical):
        for j in range(margem_horizontal, largura_imagem - margem_horizontal):
            # Extraia a região da imagem correspondente ao kernel
            regiao = imagem[i - margem_vertical:i + margem_vertical + 1, j - margem_horizontal:j + margem_horizontal + 1]

            # Aplique o kernel à região e some os produtos
            valor_pixel = np.sum(regiao * kernel)

            # Atribua o valor resultante à imagem de saída
            imagem_resultante[i, j] = valor_pixel

    return imagem_resultante

In [83]:
import numpy as np

def convolucao_manual_colorida(imagem, kernel):
    altura_imagem = imagem.shape[0]
    largura_imagem = imagem.shape[1]
    canais = imagem.shape[2]
    altura_kernel, largura_kernel = kernel.shape

    # Inicialize uma matriz para a imagem resultante
    imagem_resultante = np.zeros_like(imagem)

    # Calcule as margens para garantir que o kernel seja aplicado corretamente
    margem_vertical = altura_kernel // 2
    margem_horizontal = largura_kernel // 2

    # Realize a convolução para cada canal de cor
    for canal in range(canais):
        for i in range(margem_vertical, altura_imagem - margem_vertical):
            for j in range(margem_horizontal, largura_imagem - margem_horizontal):
                # Extraia a região da imagem correspondente ao kernel para o canal atual
                regiao = imagem[i - margem_vertical:i + margem_vertical + 1, j - margem_horizontal:j + margem_horizontal + 1, canal]

                # Aplique o kernel à região e some os produtos
                valor_pixel = np.sum(regiao * kernel)

                # Atribua o valor resultante à imagem de saída para o canal atual
                imagem_resultante[i, j, canal] = valor_pixel

    return imagem_resultante

In [113]:
img_gray = cv2.imread('../dataset/OperacaoPontuais_FiltroEspacial/data/cmyk_paint.png',0)
img = cv2.imread('../dataset/OperacaoPontuais_FiltroEspacial/data/cmyk_paint.png',1)
# Defina um kernel de suavização (filtro média 3x3)
kernel = np.array([[-1, -1, -1],
                  [-1, 8, -1],
                  [-1, -1, -1]])

# Aplique a convolução
imagem_resultante_cinza = convolucao_manual_cinza(img_gray, kernel)
imagem_resultante_col = convolucao_manual_colorida(img, kernel)

cv2.imshow('imagem_resultante_cinza', imagem_resultante_cinza)
cv2.imshow('imagem_resultante_col', imagem_resultante_col)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [114]:
# Defina a média e o desvio padrão para o ruído gaussiano
media = 0
desvio_padrao = 0.5

# Crie uma matriz com ruído gaussiano
ruido = np.random.normal(media, desvio_padrao, img.shape).astype(np.uint8)

# Adicione o ruído à imagem
imagem_ruidosa = cv2.add(img, ruido)

# Aplique filtros de suavização
suavizacao_media_3x3 = cv2.blur(imagem_ruidosa, (3, 3))
suavizacao_media_5x5 = cv2.blur(imagem_ruidosa, (5, 5))
suavizacao_media_7x7 = cv2.blur(imagem_ruidosa, (7, 7))

sigma = 1  # Valor do desvio padrão para o filtro gaussiano

suavizacao_gaussiana_3x3 = cv2.GaussianBlur(imagem_ruidosa, (3, 3), sigma)
suavizacao_gaussiana_5x5 = cv2.GaussianBlur(imagem_ruidosa, (5, 5), sigma)
suavizacao_gaussiana_7x7 = cv2.GaussianBlur(imagem_ruidosa, (7, 7), sigma)

suavizacao_mediana_3x3 = cv2.medianBlur(imagem_ruidosa, 3)
suavizacao_mediana_5x5 = cv2.medianBlur(imagem_ruidosa, 5)
suavizacao_mediana_7x7 = cv2.medianBlur(imagem_ruidosa, 7)

# Exiba as imagens suavizadas
cv2.imshow('Suavizacao Media 3x3', suavizacao_media_3x3)
cv2.imshow('Suavizacao Media 5x5', suavizacao_media_5x5)
cv2.imshow('Suavizacao Media 7x7', suavizacao_media_7x7)
cv2.imshow('Suavizacao Gaussiana 3x3', suavizacao_gaussiana_3x3)
cv2.imshow('Suavizacao Gaussiana 5x5', suavizacao_gaussiana_5x5)
cv2.imshow('Suavizacao Gaussiana 7x7', suavizacao_gaussiana_7x7)
cv2.imshow('Suavizacao Mediana 3x3', suavizacao_mediana_3x3)
cv2.imshow('Suavizacao Mediana 5x5', suavizacao_mediana_5x5)
cv2.imshow('Suavizacao Mediana 7x7', suavizacao_mediana_7x7)

# Aguarde o usuário pressionar uma tecla e depois feche as janelas
cv2.waitKey(0)
cv2.destroyAllWindows()

### Sharpening

4. Implemente os filtros de sharpening, utilizando as diferentes mascarás apresentadas no livro de referência da disciplina:
 - Laplaciano (figura 3.37)
 - Sharpening implementado na figura 3.38
 - Unsharp Masking e Highboost Filtering
 - Implemente os detectores de borda Laplaciano, Roberts e Sobel
 - Utilize a função Canny da OpenCV na mesma imagem usando nos outros detectores
 
5. Relate as observações visíveis nas imagens de saída com os filtros usados, descrevendo o comportamento visual em relação as bordas, ruídos e esmaecimento da imagem com os diferentes kernels e para os diferentes algoritmos
6. Você observa uma discrepante melhoria na detecção de bordas pelo Canny em comparação aos outros 3 que justifique a complexidade adicional?
7. Aplique o filtro Sobel pós operação com um filtro de suavização (a sua escolha) e compare com a saída do Canny. Descreva suas observações técnicas sobre o comportamento.